In [2]:
import numpy as np
import os
import pandas as pd
from pathlib import Path

In [3]:
base_root = Path(os.path.abspath('')).resolve().parent
file_path = base_root / 'raw' / '20220119' / 'R021_DEMO_Batch_EDC' / 'R021 EDC.xlsx'

# define sheets names
sheet_names = {
    'PICO': 'OLXPICO03',
    'PIPR': 'OLXPIPK03',
    'PIPB': 'OLXPIPB03',
    'BOOX': 'TPAB04 SPOV',
    'LCDP': 'TPAB04 LCDS',
    'ODFX': 'TPAB04 ODFX01',
    'SEUV': 'TPAB04 SEUV',
    'SEPB': 'TPAB04 SEPB',
}
base = {
        'ID'           : 'GLASS_ID', 
        'Output Time'  : 'TRANSDT',
}
data_structure = {
    'PICO': base | {
        'Speed: Anilox': 'Anilox_speed_EDC', # 匹配速度：A 輪
        'Speed: Table' : 'TABLE_SPEED_EDC',  # 匹配速度：Table
        'PI flow'      : 'PI_flow_EDC',      # PI 吐出量
    },
    'PIPR': base | {
        'Temp'         : 'HeaterTemp_EDC',
        'Time'         : 'Prebake_time_EDC',
    },
    'PIPB': base | {
        'Temp'         : ['MaxTemp_EDC', 'MinTemp_EDC'],
        'Time'         : 'Bake_Time_EDC',
    },
    'BOOX': base | {
        'Temp'         : 'Temp_EDC',
        'Time'         : ['Input_Time_EDC', 'Output_Time_EDC'],
    },
    'LCDP': base | {
        'LC drop qty'  : ['1St_Dp_W_001'],
    },
    'ODFX': base | {
        'Pair CF ID'   : 'CF_Work_ID_EDC',
        'Product ID'   : 'PRODUCT_ID',
    },
    'SEUV': base | {
        'Exposure qty' : { # 積光量 = time * power
            'time' : 'Irradia_Time_EDC', 
            'power': ['Lamp1_Lumin_EDC', 'Lamp2_Lumin_EDC']
        }, 
    },
    'SEPB': base | {
        'Temp'         : 'OvenRealTemp_EDC',
        'Time'         : 'Real_Bake_Tm_EDC',
    },
}

In [125]:
test_tf_df = pd.merge(tfcf, pico, left_on='TFT ID', right_on='GLASS_ID', how='left').head()
test_tf_df = test_tf_df.rename(lambda name: 'TF_' + name, axis='columns')
test_cf_df = pd.merge(tfcf, pico, left_on='CF ID', right_on='GLASS_ID', how='left').head()
test_cf_df = test_cf_df.rename(lambda name: 'CF_' + name, axis='columns')

In [4]:
# Loading data
raw = { station: pd.read_excel(file_path, station) for station in data_structure }

In [5]:
# data preporcessing
clean = {}

# Flat all the values, the value should be the fields of raw data.
def flat_list(x) -> list:
    result = []
    if type(x) == list:
        for item in x:
            result += flat_list(item)
        return result
    elif type(x) == dict:
        for value in x.values():
            result += flat_list(value)
        return result
    else:
        return [x]

for station, fields in data_structure.items():
    columns = flat_list(fields) 
    clean[station] = (
        raw[station][columns]
            .dropna()
            .sort_values(base["Output Time"])
            .drop_duplicates(base["ID"], keep='last')
            .reset_index(drop=True)
    )

In [6]:
# data calculation
# calculate some fields with multiple value
clean['ODFX'].columns = ['ODFX: ' + field for field in data_structure['ODFX']]
clean['PICO'].columns = ['PICO: ' + field for field in data_structure['PICO']]
clean['PIPR'].columns = ['PIPR: ' + field for field in data_structure['PIPR']]

clean['PIPB']['PIPB: Temp'] = clean['PIPB'][data_structure['PIPB']['Temp']].mean(axis=1)
clean['PIPB'] = clean['PIPB'].rename({
    base['ID']: 'PIPB: ID',
    base['Output Time']: 'PIPB: Output Time',
    data_structure['PIPB']['Time']: 'PIPB: Time'
}
, axis='columns')[['PIPB: ID', 'PIPB: Output Time', 'PIPB: Temp', 'PIPB: Time']]

In [45]:
# recombine the table
# 1. TFT and CF separately combine
## 1. TFT


In [60]:
clean['PIPB']['Temp'] = clean['PIPB'][data_structure['PIPB']['Temp']].mean(axis=1)

In [62]:
clean['PIPB'].rename({
        base['ID']: 'PIPB: ID',
        base['Output Time']: 'PIPB: Output Time',
        data_structure['PIPB']['Time']: 'PIPB: Time'
    }
    , axis='columns')

,PIPB: ID,PIPB: Output Time,MaxTemp_EDC,MinTemp_EDC,PIPB: Time,Temp
0,T11C1018NE,2022-01-10 21:36:43,235,234,937,234.5
1,T11C1018NH,2022-01-10 21:37:27,235,235,945,235.0
2,T11C1020NA,2022-01-10 21:38:49,235,234,927,234.5
3,T11C1020NC,2022-01-10 21:43:59,235,235,904,235.0
4,T11C1018NG,2022-01-10 21:44:43,235,234,904,234.5
...,...,...,...,...,...,...
234,T11BP016NU,2022-01-11 03:31:54,235,235,906,235.0
235,T11BP019NB,2022-01-11 03:32:32,235,235,906,235.0
236,T11BP016NP,2022-01-11 03:33:45,235,234,906,234.5
237,T11BP014NV,2022-01-11 03:34:23,235,235,906,235.0


In [22]:
clean['BOOX']['TRANSDT']

0     2022-01-11 10:18:41
1     2022-01-11 12:44:25
2     2022-01-11 13:08:56
3     2022-01-11 13:13:03
4     2022-01-11 13:19:12
              ...        
115   2022-01-12 10:34:23
116   2022-01-12 10:40:52
117   2022-01-12 10:47:22
118   2022-01-12 10:53:53
119   2022-01-12 11:00:23
Name: TRANSDT, Length: 120, dtype: datetime64[ns]

In [11]:
'20' + str(clean['BOOX']['Input_Time_EDC'][0])

'20220111100656'

In [8]:
clean['PIPB']

,PIPB: ID,PIPB: Output Time,PIPB: Temp,PIPB: Time
0,T11C1018NE,2022-01-10 21:36:43,234.5,937
1,T11C1018NH,2022-01-10 21:37:27,235.0,945
2,T11C1020NA,2022-01-10 21:38:49,234.5,927
3,T11C1020NC,2022-01-10 21:43:59,235.0,904
4,T11C1018NG,2022-01-10 21:44:43,234.5,904
...,...,...,...,...
234,T11BP016NU,2022-01-11 03:31:54,235.0,906
235,T11BP019NB,2022-01-11 03:32:32,235.0,906
236,T11BP016NP,2022-01-11 03:33:45,234.5,906
237,T11BP014NV,2022-01-11 03:34:23,235.0,906
